# Overview of database files and file groups

In [33]:
SELECT
    db.name AS DBName,
    mf.type_desc AS FileType,
    ds.name AS FileGroup,
    Physical_Name AS Location
FROM
    sys.master_files mf
INNER JOIN sys.databases db ON db.database_id = mf.database_id
INNER JOIN sys.data_spaces ds ON ds.data_space_id = mf.data_space_id
WHERE db.name = 'WideWorldImporters'


(5 rows affected)

Total execution time: 00:00:00.051

DBName FileType FileGroup Location WideWorldImporters ROWS PRIMARY C:\Program Files\Microsoft SQL Server\MSSQL13.MSSQLSERVER\MSSQL\DATA\WideWorldImporters.mdf WideWorldImporters ROWS USERDATA C:\Program Files\Microsoft SQL Server\MSSQL13.MSSQLSERVER\MSSQL\DATA\WideWorldImporters_UserData.ndf WideWorldImporters ROWS SECONDARY C:\Program Files\Microsoft SQL Server\MSSQL13.MSSQLSERVER\MSSQL\DATA\SECONDARY_DATA1.ndf WideWorldImporters ROWS SECONDARY C:\Program Files\Microsoft SQL Server\MSSQL13.MSSQLSERVER\MSSQL\DATA\SECONDARY_DATA2.ndf WideWorldImporters FILESTREAM WWI_InMemory_Data C:\Program Files\Microsoft SQL Server\MSSQL13.MSSQLSERVER\MSSQL\DATA\WideWorldImporters_InMemory_Data_1

# Delete Scheme and Function to start over

In [32]:
IF EXISTS (SELECT * FROM sys.partition_schemes WHERE name = 'PS_OrderDate')  DROP PARTITION SCHEME [PS_OrderDate];
GO 
IF EXISTS (SELECT * FROM sys.partition_functions WHERE name = 'PF_OrderDate')  DROP PARTITION FUNCTION [PF_OrderDate];
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.018

# Create Partition Function and Scheme

In [8]:
-- Create Partition Function dynamically from 2019-08-01 to three months ago
DECLARE @WorkingDate DATE;
DECLARE @DateString NVARCHAR(MAX);
DECLARE @PartitionCountPrimary INT;
DECLARE @PartitionCountSecondary INT;
DECLARE @sql_function NVARCHAR(max);

SET @WorkingDate = '20181001';
SET @DateString = N'';
SET @PartitionCountPrimary = 0;
SET @PartitionCountSecondary = 0;

-- Loop per month
WHILE @WorkingDate < DATEADD(MONTH, -3, CONVERT(DATE, FORMAT(GETDATE(), 'yyyy-MM-01'))) -- Set the number of months to create monthly partitions
BEGIN
    SET @DateString = @DateString + CONVERT(NVARCHAR(8), @WorkingDate, 112) + N', ';
	SET @PartitionCountSecondary = @PartitionCountSecondary + 1;
    SET @WorkingDate = DATEADD(MONTH, 1, @WorkingDate);
END;

-- Loop per day from the last three months
WHILE @WorkingDate <= CONVERT(DATE, GETDATE()) -- Loop through single days
BEGIN
    SET @DateString = @DateString + CONVERT(NVARCHAR(8), @WorkingDate, 112) + N', ';
	SET @PartitionCountPrimary = @PartitionCountPrimary + 1;
    SET @WorkingDate = DATEADD(DAY, 1, @WorkingDate);
END;

SET @DateString = @DateString + CONVERT(NVARCHAR(8), @WorkingDate, 112);
SET @PartitionCountPrimary = @PartitionCountPrimary + 1;

SET @sql_function = 'CREATE PARTITION FUNCTION [PF_OrderDate] (INT) AS RANGE RIGHT FOR VALUES (' + @DateString + ')'

--PRINT @sql_function
EXEC sp_executesql @sql_function;

-- Create Partition Scheme dynamically based on the Partition Function (canr be replaced by "ALL TO" when all partitions need to be in one filegroup!)
DECLARE @sql_scheme NVARCHAR(MAX);
DECLARE @loopCount INT = 0;

SET @sql_scheme = 'CREATE PARTITION SCHEME PS_OrderDate AS PARTITION [PF_OrderDate] TO ('

WHILE @loopCount < @PartitionCountSecondary
BEGIN
	
	SET @sql_scheme = @sql_scheme + '[USERDATA], ';
	
	SET @loopCount = @loopCount + 1;
END
SET @sql_scheme = @sql_scheme + '[USERDATA], ' -- Last one without a trailing comma

SET @loopCount = 0;
WHILE @loopCount < @PartitionCountPrimary
BEGIN
	
	SET @sql_scheme = @sql_scheme + '[PRIMARY], ';
	
	SET @loopCount = @loopCount + 1;
END
SET @sql_scheme = @sql_scheme + '[PRIMARY]);' -- Last one without a trailing comma

PRINT @sql_function
PRINT @sql_scheme

EXEC sp_executesql @sql_scheme;
GO

CREATE PARTITION FUNCTION [PF_OrderDate] (INT) AS RANGE RIGHT FOR VALUES (20181001, 20181101, 20181201, 20190101, 20190201, 20190301, 20190401, 20190501, 20190601, 20190701, 20190801, 20190901, 20191001, 20191101, 20191102, 20191103, 20191104, 20191105, 20191106, 20191107, 20191108, 20191109, 20191110, 20191111, 20191112, 20191113, 20191114, 20191115, 20191116, 20191117, 20191118, 20191119, 20191120, 20191121, 20191122, 20191123, 20191124, 20191125, 20191126, 20191127, 20191128, 20191129, 20191130, 20191201, 20191202, 20191203, 20191204, 20191205, 20191206, 20191207, 20191208, 20191209, 20191210, 20191211, 20191212, 20191213, 20191214, 20191215, 20191216, 20191217, 20191218, 20191219, 20191220, 20191221, 20191222, 20191223, 20191224, 20191225, 20191226, 20191227, 20191228, 20191229, 20191230, 20191231, 20200101, 20200102, 20200103, 20200104, 20200105, 20200106, 20200107, 20200108, 20200109, 20200110, 20200111, 20200112, 20200113, 20200114, 20200115, 20200116, 20200117, 20200118, 20200119, 20200120, 20200121, 20200122, 20200123, 20200124, 20200125, 20200126, 20200127, 20200128, 20200129, 20200130, 20200131, 20200201, 20200202, 20200203, 20200204, 20200205, 20200206, 20200207, 20200208, 20200209, 20200210, 20200211, 20200212)

CREATE PARTITION SCHEME PS_OrderDate AS PARTITION [PF_OrderDate] TO ([USERDATA], [USERDATA], [USERDATA], [USERDATA], [USERDATA], [USERDATA], [USERDATA], [USERDATA], [USERDATA], [USERDATA], [USERDATA], [USERDATA], [USERDATA], [USERDATA], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY], [PRIMARY]);

Das Partitionsschema "PS_OrderDate" wurde erfolgreich erstellt. "PRIMARY" ist als nächste verwendete Dateigruppe im Partitionsschema "PS_OrderDate" markiert.

Total execution time: 00:00:00.015

# Delete and Create Clustered Index on partition-column

In [9]:
IF EXISTS (SELECT * FROM sys.indexes WHERE name = 'IX_Sales_Orders_OrderDate') DROP INDEX IX_Sales_Orders_OrderDate ON Sales.Orders_Part;
GO
CREATE CLUSTERED INDEX IX_Sales_Orders_OrderDate
  ON Sales.Orders_Part (OrderDate_int)
  WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF,
         ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON)
  ON PS_OrderDate(OrderDate_int)

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.462

### Use this code to unpartition (to "reset" the test)

In [5]:
CREATE CLUSTERED INDEX IX_Sales_Orders_OrderDate
    ON Sales.Orders_Part (OrderDate)
    WITH (DROP_EXISTING = ON)
    ON [PRIMARY];
GO

Commands completed successfully.

Total execution time: 00:00:00.312

# Step 1: A different table on the slow partition needs to be created to move all data from three months ago into one partition

## Step 1a: Create table

In [28]:
DROP TABLE IF EXISTS [Sales].[Orders_Part_Merge];
CREATE TABLE [Sales].[Orders_Part_Merge](
	[OrderID] [int] NOT NULL,
	[CustomerID] [int] NOT NULL,
	[SalespersonPersonID] [int] NOT NULL,
	[PickedByPersonID] [int] NULL,
	[ContactPersonID] [int] NOT NULL,
	[BackorderOrderID] [int] NULL,
	[OrderDate] [date] NOT NULL,
	[ExpectedDeliveryDate] [date] NOT NULL,
	[CustomerPurchaseOrderNumber] [nvarchar](20) NULL,
	[IsUndersupplyBackordered] [bit] NOT NULL,
	[Comments] [nvarchar](max) NULL,
	[DeliveryInstructions] [nvarchar](max) NULL,
	[InternalComments] [nvarchar](max) NULL,
	[PickingCompletedWhen] [datetime2](7) NULL,
	[LastEditedBy] [int] NOT NULL,
	[LastEditedWhen] [datetime2](7) NOT NULL,
	[OrderDate_int] AS CONVERT(INT, CONVERT(VARCHAR(8), OrderDate, 112)) PERSISTED
)
ON [USERDATA]
GO

Commands completed successfully.

Total execution time: 00:00:00.020

## Step 1b: Move data from daily partitions into the other table

In [16]:
DECLARE @RangeFrom DATE;
DECLARE @RangeTo DATE;
DECLARE @WorkingDate DATE;

SET @RangeFrom = (SELECT TOP(1) CONVERT(DATE, FORMAT(DATEADD(MONTH, -3, GETDATE()), 'yyyy-MM') + '-01'));
SET @RangeTo = DATEADD(MONTH, 1, @RangeFrom);
SET @WorkingDate = @RangeFrom;

PRINT 'Range from: ' + CONVERT(varchar(10), @RangeFrom);
PRINT 'Range to: ' + CONVERT(varchar(10), @RangeTo);

WHILE @WorkingDate < @RangeTo
BEGIN
	
	-- Insert into the table
	INSERT INTO	[Sales].[Orders_Part_Merge] WITH (TABLOCKX)
    (
       [OrderID]
      ,[CustomerID]
      ,[SalespersonPersonID]
      ,[PickedByPersonID]
      ,[ContactPersonID]
      ,[BackorderOrderID]
      ,[OrderDate]
      ,[ExpectedDeliveryDate]
      ,[CustomerPurchaseOrderNumber]
      ,[IsUndersupplyBackordered]
      ,[Comments]
      ,[DeliveryInstructions]
      ,[InternalComments]
      ,[PickingCompletedWhen]
      ,[LastEditedBy]
      ,[LastEditedWhen]
    )

	SELECT [OrderID]
      ,[CustomerID]
      ,[SalespersonPersonID]
      ,[PickedByPersonID]
      ,[ContactPersonID]
      ,[BackorderOrderID]
      ,[OrderDate]
      ,[ExpectedDeliveryDate]
      ,[CustomerPurchaseOrderNumber]
      ,[IsUndersupplyBackordered]
      ,[Comments]
      ,[DeliveryInstructions]
      ,[InternalComments]
      ,[PickingCompletedWhen]
      ,[LastEditedBy]
      ,[LastEditedWhen] FROM [Sales].[Orders_Part] 
	WHERE CONVERT(DATE, [OrderDate]) = @WorkingDate

    PRINT 'Inserted for ' + CONVERT(varchar(10), @WorkingDate);

	SET @WorkingDate = DATEADD(DAY, 1, @WorkingDate);
END
GO

## Step 1c: Check content of the Merge table

In [17]:
SELECT OrderDate, count(*)
FROM [Sales].[Orders_Part_Merge]
GROUP BY OrderDate
ORDER BY OrderDate

# Step 2: Merge partition function

In [18]:
DECLARE @RangeFrom DATE;
DECLARE @RangeTo DATE;
DECLARE @WorkingDate DATE;

SET @RangeFrom = (SELECT TOP(1) CONVERT(DATE, FORMAT(DATEADD(MONTH, -3, GETDATE()), 'yyyy-MM') + '-02'));
SET @RangeTo = DATEADD(DAY, -1, DATEADD(MONTH, 1, @RangeFrom));
SET @WorkingDate = @RangeFrom;

PRINT 'Range from: ' + CONVERT(varchar(10), @RangeFrom);
PRINT 'Range to: ' + CONVERT(varchar(10), @RangeTo);

-- Merge partitions
WHILE @WorkingDate < @RangeTo
BEGIN

	--PRINT @WorkingDate
	ALTER PARTITION FUNCTION PF_OrderDate() MERGE RANGE (CONVERT(INT, CONVERT(nvarchar(8), @WorkingDate, 112)));

	SET @WorkingDate = DATEADD(DAY, 1, @WorkingDate);

END
GO

2019-11-02

2019-12-01

Total execution time: 00:00:00.824

# Step 3: Delete from fast storage through a temp-table

## Step 3a: Create temp-table on PRIMARY

In [27]:
DROP TABLE IF EXISTS [Sales].[Orders_Part_Temp];
CREATE TABLE [Sales].[Orders_Part_Temp](
	[OrderID] [int] NOT NULL,
	[CustomerID] [int] NOT NULL,
	[SalespersonPersonID] [int] NOT NULL,
	[PickedByPersonID] [int] NULL,
	[ContactPersonID] [int] NOT NULL,
	[BackorderOrderID] [int] NULL,
	[OrderDate] [date] NOT NULL,
	[ExpectedDeliveryDate] [date] NOT NULL,
	[CustomerPurchaseOrderNumber] [nvarchar](20) NULL,
	[IsUndersupplyBackordered] [bit] NOT NULL,
	[Comments] [nvarchar](max) NULL,
	[DeliveryInstructions] [nvarchar](max) NULL,
	[InternalComments] [nvarchar](max) NULL,
	[PickingCompletedWhen] [datetime2](7) NULL,
	[LastEditedBy] [int] NOT NULL,
	[LastEditedWhen] [datetime2](7) NOT NULL,
	[OrderDate_int] AS CONVERT(INT, CONVERT(VARCHAR(8), OrderDate, 112)) PERSISTED
)
ON [PRIMARY]
GO
-- Clustered index should not be partitioned!!
CREATE CLUSTERED INDEX IX_Sales_Orders_Temp_OrderDate ON Sales.Orders_Part_Temp (OrderDate_int)
	WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF,
			ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON);


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.014

## Step 3b: Switch partition to temp-table

In [21]:
DECLARE @PartitionId INT;
DECLARE @RightBoundary INT;

SET @RightBoundary = (SELECT TOP(1) CONVERT(int, CONVERT(nvarchar(8), DATEADD(month, DATEDIFF(month, 0, DATEADD(MONTH, -2, GETDATE())), 0), 112)));

SET @PartitionId = (SELECT TOP (1) p.partition_number
					FROM sys.partition_functions AS pf
					JOIN sys.partition_schemes AS ps ON ps.function_id = pf.function_id
					JOIN sys.indexes AS si ON si.data_space_id = ps.data_space_id
					JOIN sys.partitions AS p ON  si.object_id = p.object_id AND si.index_id = p.index_id
					LEFT JOIN sys.partition_range_values rv ON  pf.function_id = rv.function_id
															AND p.partition_number = rv.boundary_id
					WHERE pf.name = 'PF_OrderDate'
					AND rv.value = @RightBoundary)

PRINT 'Switching Partition ID  ' + CONVERT(varchar(10), @PartitionId); 

ALTER TABLE [Sales].[Orders_Part] SWITCH PARTITION @PartitionId TO [Sales].[Orders_Part_Temp];

15

Total execution time: 00:00:00.072

## Step 3c: Truncate temp-table

In [22]:
TRUNCATE TABLE [Sales].[Orders_Part_Temp];
GO

Commands completed successfully.

Total execution time: 00:00:00.010

# Step 4: Move the partition from three months ago to the SECONDARY filegroup

In [23]:
DECLARE @RightBoundary INT;
SET @RightBoundary = (SELECT TOP(1) CONVERT(int, CONVERT(nvarchar(8), DATEADD(month, DATEDIFF(month, 0, DATEADD(MONTH, -3, GETDATE())), 0), 112)));

print @RightBoundary

ALTER PARTITION FUNCTION PF_OrderDate() MERGE RANGE(@RightBoundary);

ALTER PARTITION SCHEME PS_OrderDate NEXT USED [USERDATA];

ALTER PARTITION FUNCTION PF_OrderDate() SPLIT RANGE(@RightBoundary);
GO

20191101

Total execution time: 00:00:00.029

# Step 5: Move the data from the Merge-table into the main table (on SECONDARY, monthly partition)

## Step 5a: Put a clustered index on the Merge-table in order to switch

In [25]:
CREATE CLUSTERED INDEX IX_Sales_Orders_Part_Merge_OrderDate
  ON Sales.Orders_Part_Merge (OrderDate_int)
  WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF,
         ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON)
  ON PS_OrderDate(OrderDate_int)

Commands completed successfully.

Total execution time: 00:00:00.149

## Step 5b: Switch the partition from the merge table to the main table

In [26]:
DECLARE @PartitionId INT;
DECLARE @RightBoundary INT;

SET @RightBoundary = (SELECT TOP(1) CONVERT(int, CONVERT(nvarchar(8), DATEADD(month, DATEDIFF(month, 0, DATEADD(MONTH, -2, GETDATE())), 0), 112)));

SET @PartitionId = (SELECT TOP (1) p.partition_number
					FROM sys.partition_functions AS pf
					JOIN sys.partition_schemes AS ps ON ps.function_id = pf.function_id
					JOIN sys.indexes AS si ON si.data_space_id = ps.data_space_id
					JOIN sys.partitions AS p ON  si.object_id = p.object_id AND si.index_id = p.index_id
					LEFT JOIN sys.partition_range_values rv ON  pf.function_id = rv.function_id
															AND p.partition_number = rv.boundary_id
					WHERE pf.name = 'PF_OrderDate'
					AND rv.value = @RightBoundary)

PRINT 'Switching Partition ID  ' + CONVERT(varchar(10), @PartitionId); 

ALTER TABLE [Sales].[Orders_Part_Merge] SWITCH PARTITION @PartitionId TO [Sales].[Orders_Part] PARTITION @PartitionId;

15

Total execution time: 00:00:00.166

# Cleanup

In [31]:
DROP TABLE IF EXISTS [Sales].[Orders_Part_Merge];
DROP TABLE IF EXISTS [Sales].[Orders_Part_Temp];

-- Unpartition table
CREATE CLUSTERED INDEX IX_Sales_Orders_OrderDate
    ON Sales.Orders_Part (OrderDate)
    WITH (DROP_EXISTING = ON)
    ON [PRIMARY];
GO

Commands completed successfully.

Total execution time: 00:00:00.254